<a href="https://colab.research.google.com/github/anamunoz01/DEEP_LEARNING_4_UFV/blob/main/DL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
#keras.version_
import pandas as pd
df = pd.read_csv('hola.csv')

In [ ]:
df

In [ ]:
df1 = df[['sourceText','translatedText']]

In [ ]:
df1

,sourceText,translatedText
0,Kaisa-Leena Mäkäräinen (born 11 January 1983) ...,Kaisa-Leena Mäkäräinen (nacida el 11 de enero ...
1,"Outside sports, Mäkäräinen is currently studyi...","Además de los deportes, estudia actualmente en..."
2,"Her team coach is Jonne Kähkönen, while Jarmo ...","El entrenador de su equipo es Jonne Kähkönen, ..."
3,Mäkäräinen was originally a cross-country skie...,Mäkäräinen era originalmente esquiadora de cam...
4,She started training for the biathlon in 2003.,Comenzó a entrenar para el biatlón en 2003.
...,...,...
1466,Speaking to Madrid-based Diario AS in 2013 abo...,"En 2013, habló de sus primeras frustraciones c..."
1467,Rossell proceeded to try again first under San...,Rossell volvió a intentarlo con el sucesor de ...
1468,"In the documentary ""Un Sueño Real"", she reveal...","En el documental «Un sueño real», contó que se..."
1469,Her struggle proved unsuccessful.,Su intento fue en vano.


In [ ]:
df1['translatedText'] = df1['translatedText'].apply(lambda x: "[start] " + x + " [end]")


In [ ]:
lista = df1.to_numpy()
lista = lista.tolist

In [ ]:
lista = lista.tolist()
lista

In [ ]:
import random
print(random.choice(lista))

['Despite finding most of these zoology lectures "quite boring", his interest was piqued by the neurophysiology course taught by Mitsuo Tamashige.', '[start] A pesar de que estas clases de zoología le resultaban "bastante aburridas", un curso de Neurofisiología dictado por Mitsuo Tamashige hizo que aumentara su interés. [end]']


In [ ]:
import random
random.shuffle(lista)
# 15% validación
num_val_samples = int(0.15 * len(lista))
# 70% entrenamiento
num_train_samples = len(lista) - 2 * num_val_samples
train_pairs = lista[:num_train_samples]
val_pairs = lista[num_train_samples:num_train_samples + num_val_samples]
# 15% test o pruebas
test_pairs = lista[num_train_samples + num_val_samples:]

In [ ]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

# Prepara una función de estandarización de cadenas 
# personalizada para la capa TextVectorization en 
# español: conserva [ y ] pero elimina ¿ (así como 
# todos los demás caracteres de cadenas.puntuación)
strip_chars = string.punctuation + "¿"
print(strip_chars)
strip_chars = strip_chars.replace("[", "")
print(strip_chars)
strip_chars = strip_chars.replace("]", "")
print(strip_chars)

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

# Para mantener las cosas simples, solo veremos las 
# 15.000 palabras principales en cada idioma y 
# restringiremos las oraciones a 20 palabras.
vocab_size = 15000
sequence_length = 20

# La capa en Inglés
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
# La capa en Español
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    # Generamos oraciones en español que tengan un token 
    # adicional, ya que necesitaremos compensar la oración 
    # en un paso durante el entrenamiento
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
# Aprende el vocabulario de cada idioma
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~¿
!"#$%&'()*+,-./:;<=>?@\]^_`{|}~¿
!"#$%&'()*+,-./:;<=>?@\^_`{|}~¿


In [ ]:
print(random.choice(train_spanish_texts))

[start] Obtuvo una Licenciatura en Sociología y Estudios Sociales (y un título profesional de trabajo social) en la Universidad de Keele, Reino Unido, en 1980, un Máster en ciencia en [end]


In [ ]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        # La oración de entrada en español 
        # no incluye el último token para 
        # mantener las entradas y los 
        # objetivos en la misma longitud.
        "spanish": spa[:, :-1],
    # La frase objetivo en español está un 
    # paso por delante. Ambos siguen siendo 
    # de la misma longitud (20 palabras)
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    # Utilizamos el almacenamiento en caché en memoria 
    # para acelerar el preprocesamiento
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


## **FUNCIONES SUELTAS**

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding 
        # será 2D, pero la capa de atención espera 
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para 
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [ ]:
# Forma de la entrada: (batch_size, sequence_length, embedding_dim)
def layer_normalization(batch_of_sequences):
    # Para calcular la media y la varianza, solo 
    # agrupamos datos sobre el último eje (eje -1)
    mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)
    variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
    return (batch_of_sequences - mean) / variance

In [ ]:
from tensorflow.keras import layers

In [ ]:
# Forma de la entrada: (batch_size, height, width, channels)
def batch_normalization(batch_of_images):
    # Agrupa los datos sobre el eje del lote (eje 0), 
    # lo que crea interacciones entre las muestras en un lote.
    mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2))
    variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2))
    return (batch_of_images - mean) / variance

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # Este atributo asegura que la capa propagará 
        # su máscara de entrada a sus salidas; el 
        # enmascaramiento en Keras es explícitamente 
        # opt-in. Si pasa una máscara a una capa que 
        # no implementa compute_mask() y que no expone 
        # este atributo support_masking, es un error.
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [ ]:
def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        # Genere una matriz de forma (sequence_length, sequence_length) 
        # con 1 en una mitad y 0 en la otra
        mask = tf.cast(i >= j, dtype="int32")
        # Lo replicamos a lo largo del eje del lote para obtener una matriz 
        # de forma  (batch_size, sequence_length, sequence_length)
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

In [ ]:
def call(self, inputs, encoder_outputs, mask=None):
        # Recupera la máscara causal
        causal_mask = self.get_causal_attention_mask(inputs)
        # Prepara la máscara de entrada (que describe las 
        # ubicaciones de relleno en la secuencia de destino)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            # Fusiona las dos máscaras juntas
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            # Pasamos la máscara causal a la primera capa de atención, 
            # que realiza la self-attention sobre la secuencia de destino.
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            # Pasamos la máscara combinada a la segunda 
            # capa de atención, que relaciona la secuencia 
            # de origen con la secuencia de destino
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**Transformer Extremo-a-Extremo**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# Codificamos la oración fuente
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# Codificamos la oración objetivo y la combinamos con la oración fuente codificada
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
# Predecimos una palabra para cada posición de salida
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
17/17 [==============================] - 62s 3s/step - loss: 7.9098 - accuracy: 0.0857 - val_loss: 6.7800 - val_accuracy: 0.1021
Epoch 2/30
17/17 [==============================] - 47s 3s/step - loss: 6.6307 - accuracy: 0.1264 - val_loss: 6.4377 - val_accuracy: 0.1612
Epoch 3/30
17/17 [==============================] - 49s 3s/step - loss: 6.2977 - accuracy: 0.1498 - val_loss: 6.2009 - val_accuracy: 0.1597
Epoch 4/30
17/17 [==============================] - 48s 3s/step - loss: 6.0015 - accuracy: 0.1671 - val_loss: 6.0327 - val_accuracy: 0.1768
Epoch 5/30
17/17 [==============================] - 49s 3s/step - loss: 5.7457 - accuracy: 0.1829 - val_loss: 5.9368 - val_accuracy: 0.1793
Epoch 6/30
17/17 [==============================] - 49s 3s/step - loss: 5.5007 - accuracy: 0.2066 - val_loss: 5.7626 - val_accuracy: 0.2029
Epoch 7/30
17/17 [==============================] - 47s 3s/step - loss: 5.3111 - accuracy: 0.2265 - val_loss: 5.9277 - val_accuracy: 0.1931
Epoch 8/30
17/17 [==

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        # Muestra el siguiente token
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convertimos la siguiente predicción del token en una 
        # cadena y la agregamos a la oración generada
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # Condición de salida
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Elizaveta Petrovna Glinka (Russian: Елизаве́та Петро́вна Гли́нка, also known as Dr. Liza (Russian: До́ктор Ли́за); 20 February 1962 – 25 December 2016) was a Russian humanitarian worker and charity activist.
[start] fue el primer ministro de la primera mujer de la banda de la salud mental de la banda de la
-
He currently sits on the board of Silkbank.
[start] se graduó en la escuela secundaria de la región [end]
-
He became a naturalized citizen in 1934.
[start] se convirtió en el primer ministro de la primera división [end]
-
Fukui was born in Osaka.
[start] fue vicepresidente del boj entre 2002 [end]
-
His time of 47.93 seconds to win the 1990 European Athletics Championships was a British record, and he also won gold at the 1990 Commonwealth Games.
[start] su madre era de la biblioteca de la biblioteca de la academia de la biblioteca de las políticas y la
-
He created history by holding three different designations in the same university as,Librarian of University of Ceylon (1971 